In [11]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate

from dotenv import find_dotenv, load_dotenv
import os
import openai
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())

openai_api_key = os.environ["OPENAI_API_KEY"]

openai_model = "gpt-4o-mini"

llm = ChatOpenAI(temperature = 0.0, model = openai_model)
creative_llm = ChatOpenAI(temperature = 0.9, model = openai_model)

article = """This is an quick summary about the {topic}"""

system_prompt = SystemMessagePromptTemplate.from_template(""" You are very helpful AI Assistant and your name is {Agent_name}""")

user_prompt = HumanMessagePromptTemplate.from_template("""Introduce your self and Please create an article about {topic}""", input_variables =["topic"])

#user_prompt.format(topic = "MS Dhoni")

myfirst_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

chain_one = (
            {"topic" : lambda x: x["topic"],
             "Agent_name" : lambda x: x["Agent_name"]} 
            | myfirst_prompt 
            | llm 
            | {"summary" : lambda x: x.content}
            )

#summary_output = chain_one.invoke({"topic": "MS Dhoni", "Agent_name" : "SKPAI"})



summary_output

{'summary': 'Hello! I am SKPAI, an AI assistant here to help you with any questions or tasks you may have. \n\n---\n\n**MS Dhoni: The Captain Cool of Indian Cricket**\n\nMahendra Singh Dhoni, popularly known as MS Dhoni, is one of the most iconic and successful cricketers in the history of Indian cricket. Born on July 7, 1981, in Ranchi, Jharkhand, Dhoni is renowned for his calm demeanor, exceptional leadership skills, and remarkable achievements on the cricket field.\n\nDhoni made his international debut for India in December 2004 in an ODI against Bangladesh and quickly rose through the ranks to become one of the most influential players in the team. He is best known for his exceptional wicket-keeping skills, lightning-fast stumpings, and his ability to finish matches with his powerful batting.\n\nOne of Dhoni\'s most significant contributions to Indian cricket was his captaincy. He took over the reins of the Indian team in 2007 and led them to numerous victories, including the ICC T

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers.pydantic import PydanticOutputParser

class myTable(BaseModel):
    Name : str = Field(description="Name of the Player")
    DOB : str = Field(description="Date of Birth")
    Country : str =Field(description="Country for which he is playing")
    Age : int = Field(description="Age of the Player")
    
""" pydantic_parser = PydanticOutputParser(pydantic_object=myTable)
format_output_instructions = pydantic_parser.get_format_instructions() """
    
player_user_prompt = HumanMessagePromptTemplate.from_template("""Introduce your self and Please create an article about {topic}""", input_variables =["topic"])

myplayer_prompt = ChatPromptTemplate.from_messages([system_prompt, player_user_prompt])

structured_llm = llm.with_structured_output(myTable)

chain_pydantic = (
                    {"topic" : lambda x: x["topic"],
                     "Agent_name" : lambda x: x["Agent_name"]}
                    | myplayer_prompt
                    | structured_llm
                    | {
                        "Name" : lambda x: x.Name,
                        "DOB" : lambda x: x.DOB,
                        "Age" : lambda x: x.Age,
                        "Country" : lambda x: x.Country
                      }
                )

response = chain_pydantic.invoke({"topic" : "SR Tendulkar", "Agent_name" : "SKPAI"})

response

{'Name': 'Sachin Ramesh Tendulkar',
 'DOB': '1973-04-24',
 'Age': 50,
 'Country': 'India'}

: 